<a href="https://colab.research.google.com/github/Jacksonngugi/sms_text_classification/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/522.5 MB 1.2 MB/s eta 0:06:21
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.10/http/client.py", line 466, in read
    s = self.fp.read(amt)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, bu

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-11-12 09:44:18--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   598KB/s    in 0.6s    

2023-11-12 09:44:20 (598 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-11-12 09:44:20--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[===========

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', names=['label','text'])
test_data = pd.read_csv(test_file_path, sep='\t', names=['label','text'])
test_data.head()

,label,text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [ ]:
spam_messages = train_data[train_data["label"] == "spam"]["text"]
ham_messages = train_data[train_data["label"] == "ham"]["text"]
print(f"Number of spam messages: {len(spam_messages)}")
print(f"Number of ham messages: {len(ham_messages)}")

Number of spam messages: 560
Number of ham messages: 3619


In [ ]:
encode = {'ham':0, 'spam':1}
train_data['label'] = train_data['label'].map(encode)
test_data['label'] = test_data['label'].map(encode)

train_label = train_data.pop('label')
test_label = test_data.pop('label')

train_data.head()


,text
0,ahhhh...just woken up!had a bad dream about u ...
1,you can never do nothing
2,"now u sound like manky scouse boy steve,like! ..."
3,mum say we wan to go then go... then she can s...
4,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
MAX = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'].values)
encoded_train_data = tokenizer.texts_to_sequences(train_data['text'].values)
encoded_test_data = tokenizer.texts_to_sequences(test_data['text'].values)
padded_train_data = sequence.pad_sequences(encoded_train_data, maxlen=MAX, padding='post')
padded_test_data = sequence.pad_sequences(encoded_test_data, maxlen=MAX, padding='post')

padded_train_data

array([[2482,   44,  142, ...,   44,   12,   14],
       [   3,   29,  281, ...,    0,    0,    0],
       [  19,    6,  831, ...,    0,    0,    0],
       ...,
       [  47,  464,  337, ...,    0,    0,    0],
       [  87, 3665, 7690, ...,  750,    2, 1113],
       [ 218,    6,  731, ..., 7692,   57,  180]], dtype=int32)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(35000, 32, input_length=MAX),
    tf.keras.layers.SimpleRNN(32, activation='relu',return_sequences=False),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 30, 32)            1120000   
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1122113 (4.28 MB)
Trainable params: 1122113 (4.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=7, verbose=1)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(padded_train_data, train_label.values, epochs=100, verbose=1, callbacks=[early_stop], validation_data=[padded_test_data, test_label])

Epoch 1/100
131/131 [==============================] - 22s 151ms/step - loss: 0.0277 - accuracy: 0.9978 - val_loss: 0.1526 - val_accuracy: 0.9871
Epoch 2/100
131/131 [==============================] - 9s 69ms/step - loss: 0.0025 - accuracy: 0.9988 - val_loss: 0.1544 - val_accuracy: 0.9856
Epoch 3/100
131/131 [==============================] - 9s 67ms/step - loss: 2.0233e-05 - accuracy: 1.0000 - val_loss: 0.1555 - val_accuracy: 0.9849
Epoch 4/100
131/131 [==============================] - 5s 40ms/step - loss: 9.0730e-06 - accuracy: 1.0000 - val_loss: 0.1598 - val_accuracy: 0.9842
Epoch 5/100
131/131 [==============================] - 6s 49ms/step - loss: 5.7610e-06 - accuracy: 1.0000 - val_loss: 0.1630 - val_accuracy: 0.9842
Epoch 6/100
131/131 [==============================] - 5s 35ms/step - loss: 4.0820e-06 - accuracy: 1.0000 - val_loss: 0.1661 - val_accuracy: 0.9849
Epoch 7/100
131/131 [==============================] - 4s 31ms/step - loss: 3.0004e-06 - accuracy: 1.0000 - val_loss: 

In [ ]:
model.evaluate(padded_test_data, test_label)

44/44 [==============================] - 0s 5ms/step - loss: 0.1730 - accuracy: 0.9842


[0.1729920655488968, 0.9841954112052917]

In [ ]:
def predict_message(pred_text):
  pred_encoded = tokenizer.texts_to_sequences([pred_text])
  pred_padded = sequence.pad_sequences(pred_encoded, maxlen=MAX, padding='post')
  prediction = model.predict(pred_padded).flatten()[0]

  if prediction < 0.5:
    return (prediction, 'ham')
  return (prediction, 'spam')

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 323ms/step
(2.448208e-09, 'ham')


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 18ms/step
You haven't passed yet. Keep trying.
